# Get universitary restaurants menus



<a href="4_Regex.html">Go back to Chapter 3</a> 

- First import module needed

In [1]:
from bs4 import BeautifulSoup
import urllib3
import re
import time
import numpy as np

- Select one page as an example, here we use gallia's restaurant


In [2]:
urlpage = 'https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-ouvert/'
urlpage

'https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-ouvert/'

- Get the html content using Beautiful soup

In [3]:
req = urllib3.PoolManager()
res = req.request('GET', urlpage)
soup = BeautifulSoup(res.data, 'html.parser')
str(soup)[:2000]

'<!DOCTYPE html>\n\n<html class="no-js" lang="fr">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>\n<!--[if lt IE 9]>\n\t\t\t<script src="http://html5shiv.googlecode.com/svn/trunk/html5.js"></script>\n\t\t<![endif]-->\n<link href="/wp-content/themes/crous/img/crous.ico" rel="shortcut icon"/>\n<link href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/leaflet.css" rel="stylesheet">\n<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/leaflet.js"></script>\n<script src="https://www.mapquestapi.com/sdk/leaflet/v2.2/mq-map.js?key=5SkA8njxRCCvI2CLsz1ek7lH4vpqypAE"></script>\n<!-- This site is optimized with the Yoast SEO plugin v4.9 - https://yoast.com/wordpress/plugins/seo/ -->\n<title>Resto U\' Esplanade - OUVERT - Crous de Strasbourg</title>\n<link href="https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-ouvert/" rel="canonical"/>\n<!-- / Yoast SEO plugin. -->\n<li

- Find out how you will get the information you want: with a right click you have previously inspect the webpage with your navigator. ```'content-repas'``` seems to be a good option to get meals in a structured way.  
```'content-repas'``` is the html class and ```'div'```is the element. See how it is written ```<div class="content-repas">```

- Have a look on the content after specifiying the type of element and class you wanted to match. There is three different meal types in a day so we know that the first three elements will constitute the first day. 


In [4]:
# Return elements that matched the html class in a list
content = soup.find_all('div' ,class_= 'content-repas')
content[:3]

[<div class="content-repas"><div><span class="name">Pas de service</span></div></div>,
 <div class="content-repas"><div><span class="name">Pôle Ici et Ailleurs</span><ul class="liste-plats"><li>UNIQUEMENT A EMPORTER</li><li></li><li>Boulettes de viande (boeuf et porc) maison</li><li>Sauce forestière</li><li>Gratin dauphinois</li><li>Brocolis</li></ul><span class="name">Pôle végétarien</span><ul class="liste-plats"><li>Colombo de légumes au lait de coco</li></ul><span class="name">Cristal Shop (RDC)</span><ul class="liste-plats"><li>UNIQUEMENT A EMPORTER</li><li></li><li>Burger du jour</li><li>Pizza du jour</li></ul><span class="name">Desserts maison</span><ul class="liste-plats"><li>Gâteau de patates douces maison</li></ul><span class="name">Pôle Pêche du jour</span><ul class="liste-plats"><li>menu non communiqué</li></ul></div></div>,
 <div class="content-repas"><div><span class="name">Pas de service</span></div></div>]

- We still not have the date for each meal in a given restaurant, we know that it's another day because there is 3 meals, but what if one is closed let's say a Satursday and show the menu of Monday directly ? I didn't think about it last time but we need to take it into account.

- Let's first create a function to get the webpage, the arguments to get a flexible function could be: 
    - url of the page
    - element of the html
    - class

In [5]:
def get_page(urlpage,element ,html_class):
    """
    From an link and a html class that you select by inspecting manually the page
    Return a list with elements that matched this class
    """
    # Get page in html
    req = urllib3.PoolManager()
    res = req.request('GET', urlpage)
    soup = BeautifulSoup(res.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    content = soup.find_all(element ,class_= html_class)
    
    return content 

In [6]:
content = get_page('http://www.crous-strasbourg.fr/restaurant/resto-u-gallia-ouvert/',
                'ul','slides')
content

[<ul class="slides">
 <li>
 <h3>Menu du jeudi 28 janvier 2021</h3>
 <div class="content clearfix">
 <div><h4>Petit déjeuner</h4><div class="content-repas"><div><span class="name">Pas de service</span></div></div></div>
 <div><h4>Déjeuner</h4><div class="content-repas"><div><span class="name">Menu étudiant</span><ul class="liste-plats"><li>Risotto au bœuf</li><li></li><li>Escalope viennoise</li><li>Semoule</li><li>Légumes</li><li></li><li>Pizza paysanne</li></ul><span class="name">Végétarien ou végétal</span><ul class="liste-plats"><li>Clafoutis aux tomates cerises</li><li>Salade verte</li><li></li><li>Tchoutchouka</li></ul><span class="name">Brasserie</span><ul class="liste-plats"><li>menu non communiqué</li></ul></div></div></div>
 <div><h4>Dîner</h4><div class="content-repas"><div><span class="name">Pas de service</span></div></div></div>
 </div>
 </li>
 <li>
 <h3>Menu du vendredi 29 janvier 2021</h3>
 <div class="content clearfix">
 <div><h4>Petit déjeuner</h4><div class="content-re


- We want everything between ```<div class="content-repas"><div>``` and the first ```</div>``` that appears, we can just catch the group of character between these two entities. Create a group with ```()``` to extract this part with ```re.findall``` :
    - This group can be composed of everything so specify a ```.```.
    - We can have 0 or more charater so put a ```*```after the ```.```.
    - Finally the ```?``` will control for the repetition ```</div>```, so when it will meet the first ```</div>```, it will stop.
 
- You will then try to find if there is some stuff in your string that match this condition. The regex will be written ```'<div class="content-repas"><div>(.*?)</div>'```. Of course it doen't mean there are no other way of doing it or even writting it, but it works and it's pretty simple to read.


In [7]:
dates = re.findall('<h3>(.*?)</h3>',str(content))
content = re.findall('<div class="content-repas"><div>(.*?)</div>',str(content))

content

['<span class="name">Pas de service</span>',
 '<span class="name">Menu étudiant</span><ul class="liste-plats"><li>Risotto au bœuf</li><li></li><li>Escalope viennoise</li><li>Semoule</li><li>Légumes</li><li></li><li>Pizza paysanne</li></ul><span class="name">Végétarien ou végétal</span><ul class="liste-plats"><li>Clafoutis aux tomates cerises</li><li>Salade verte</li><li></li><li>Tchoutchouka</li></ul><span class="name">Brasserie</span><ul class="liste-plats"><li>menu non communiqué</li></ul>',
 '<span class="name">Pas de service</span>',
 '<span class="name">Pas de service</span>',
 '<span class="name">Menu étudiant</span><ul class="liste-plats"><li>Morbiflette</li><li>Salade verte</li><li></li><li>Pad Thaï de dinde</li></ul><span class="name">Végétarien ou végétal</span><ul class="liste-plats"><li>Enchillada aux légumes et chèvre</li></ul><span class="name">Brasserie</span><ul class="liste-plats"><li>menu non communiqué</li></ul>',
 '<span class="name">Pas de service</span>']

In [8]:
dates

['Menu du jeudi 28 janvier 2021', 'Menu du vendredi 29 janvier 2021']

- It seems that we are interested by two things : 
    - Which pole is it ?  
    ```<span class="name"></span>``` gives us the name, when no place within the restaurant are open, it is written ```Pas de service```, otherwise it can be written ```Pôle Pêche du jour```for example.
    
    - What we will eat there ?      
    ```<ul class="liste-plats"></ul>``` gives us the meal list, when no place within the restaurant are open there is no ```<ul class="liste-plats">```
 
- Let's select the second meal of the day and see names that we get, by comparing with the previous cell it seems that we have everything that we need

In [9]:
names = re.findall('<span class="name">(.*?)</span>',
                   str(content[1]))
names

['Menu étudiant', 'Végétarien ou végétal', 'Brasserie']

- You can do the exact same thing to get the meals


In [10]:
meals = re.findall('<ul class="liste-plats">(.*?)</ul>',
                   str(content[1]))
meals  

['<li>Risotto au bœuf</li><li></li><li>Escalope viennoise</li><li>Semoule</li><li>Légumes</li><li></li><li>Pizza paysanne</li>',
 '<li>Clafoutis aux tomates cerises</li><li>Salade verte</li><li></li><li>Tchoutchouka</li>',
 '<li>menu non communiqué</li>']

- It's still very dirty down there, let's clean it up!

- Let's retrieve everything within ```<li></li>``` as same as before. The only differences is that we already have a list of information so we store a list in a list using list comprehension.

In [11]:
meals_list = [re.findall('<li>(.*?)</li>',item) for item in meals]
meals_list

[['Risotto au bœuf',
  '',
  'Escalope viennoise',
  'Semoule',
  'Légumes',
  '',
  'Pizza paysanne'],
 ['Clafoutis aux tomates cerises', 'Salade verte', '', 'Tchoutchouka'],
 ['menu non communiqué']]

- There is some stuffs that we don't care about, as the ```''``` or the ```'UNIQUEMENT A EMPORTER'``` etc etc. Let's remove them:
   - Create a list with words that you want to throw away
   - Filter out all words that are not in the list


In [12]:
to_remove =  ['UNIQUEMENT A EMPORTER','A EMPORTER','','-','ou']
clean_meals_list = [list(filter(lambda x: x not in to_remove,meals)) for meals in meals_list]
clean_meals_list

[['Risotto au bœuf',
  'Escalope viennoise',
  'Semoule',
  'Légumes',
  'Pizza paysanne'],
 ['Clafoutis aux tomates cerises', 'Salade verte', 'Tchoutchouka'],
 ['menu non communiqué']]

- Let's store it in a dict

In [13]:
# Create a Dict with the name of the pole as key and meal content as value
dict_content = {name : meal for name, meal in zip(names,clean_meals_list)}
dict_content

{'Menu étudiant': ['Risotto au bœuf',
  'Escalope viennoise',
  'Semoule',
  'Légumes',
  'Pizza paysanne'],
 'Végétarien ou végétal': ['Clafoutis aux tomates cerises',
  'Salade verte',
  'Tchoutchouka'],
 'Brasserie': ['menu non communiqué']}

- We do know now two things :
   - Download the content of each meal and store in a dirty list
   - Clean up individually the list to get names and meals for a given restaurant.
   
- let's create a function!
   - Get the each meal and each name for a given restaurant and for a given schedule, we just need the whole content of this given meal.  
   We will need to loop over each schedule by keeping in mind that every 3 schedules the date change.

In [14]:
def get_menu(content):
    """
    Get meal content, pole names, clean the menu and store it in a dict
    """
    # Extract first titles using the class 'name'
    names = re.findall('<span class="name">(.*?)</span>',str(content))
    
    # Extract meals with the class 'liste-plats'
    meals = re.findall('<ul class="liste-plats">(.*?)</ul>',str(content))
    
    # Select everything within 'li' tags
    meals_list = [re.findall('<li>(.*?)</li>',item) for item in meals]
    
    # Remove some words that you want to throwaway
    to_remove =  ['UNIQUEMENT A EMPORTER','A EMPORTER','','-','ou']
    clean_meals_list = [list(filter(lambda x: x not in to_remove,meals)) for meals in meals_list]
    
    # Create a Dict with the name of the pole as key and meal content as value
    dict_content = {name : meal for name, meal in zip(names,clean_meals_list)}
    
    return dict_content

- Now we need to get all restaurants urls, we will them here :
 https://www.crous-strasbourg.fr/restauration/vos-restaurants-et-leurs-menus/

- Use the get_page function, specify ```'div'``` and ```''links'``` because we want to catch up ```<div class="links">```.


In [15]:
urlpage = 'https://www.crous-strasbourg.fr/restauration/vos-restaurants-et-leurs-menus/'
   
links = get_page(urlpage,'div','links')
links[:5]

[<div class="links">
 <a class="link-marker" href="#r907">Voir sur la carte</a>
 <a class="link" href="http://www.crous-strasbourg.fr/restaurant/cybercafe-ru-illberg-ferme/">En savoir plus</a>
 </div>,
 <div class="links">
 <a class="link-marker" href="#r909">Voir sur la carte</a>
 <a class="link" href="http://www.crous-strasbourg.fr/restaurant/resto-u-gallia-ouvert/">En savoir plus</a>
 </div>,
 <div class="links">
 <a class="link-marker" href="#r1105">Voir sur la carte</a>
 <a class="link" href="http://www.crous-strasbourg.fr/restaurant/learning-center-ferme/">En savoir plus</a>
 </div>,
 <div class="links">
 <a class="link-marker" href="#r1102">Voir sur la carte</a>
 <a class="link" href="http://www.crous-strasbourg.fr/restaurant/cafeteria-le-cardo-ferme/">En savoir plus</a>
 </div>,
 <div class="links">
 <a class="link-marker" href="#r551">Voir sur la carte</a>
 <a class="link" href="http://www.crous-strasbourg.fr/restaurant/resto-u-paul-appell-ouvert/">En savoir plus</a>
 </div>]

- We see first that links that are interesting for us are opened restaurant. That's great because it's written in the url!

- Loop over links' list, check if we match ```ouvert``` in the url.
   - Capture the name of the restaurant and the appropriate link
   - Store it in a dict

In [16]:
# Create an empty dict
names_links = {}

# Iterate over each link
for link in links:
    # Check if the restaurant is open, it is written in the url !
    if bool(re.search('ouvert',str(link))):
        # Extract the name of the restaurant
        name = re.findall('restaurant/(.*)-ouvert/',str(link))[0]
        # Extract the link
        link = re.search('http:.*ouvert/',str(link)).group()
        # Update the dict
        names_links.update({name : link})

names_links

{'resto-u-gallia': 'http://www.crous-strasbourg.fr/restaurant/resto-u-gallia-ouvert/',
 'resto-u-paul-appell': 'http://www.crous-strasbourg.fr/restaurant/resto-u-paul-appell-ouvert/',
 'resto-u-esplanade': 'http://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-ouvert/',
 'resto-u-cronenbourg': 'http://www.crous-strasbourg.fr/restaurant/resto-u-cronenbourg-ouvert/',
 'resto-u-illkirch': 'http://www.crous-strasbourg.fr/restaurant/resto-u-illkirch-ouvert/',
 'resto-u-de-lillberg': 'http://www.crous-strasbourg.fr/restaurant/resto-u-de-lillberg-ouvert/',
 'resto-u-de-liut-mulhouse': 'http://www.crous-strasbourg.fr/restaurant/resto-u-de-liut-mulhouse-ouvert/',
 'resto-u-de-colmar': 'http://www.crous-strasbourg.fr/restaurant/resto-u-de-colmar-ouvert/',
 'le-cristal-shop-ru-esplanade': 'http://www.crous-strasbourg.fr/restaurant/le-cristal-shop-ru-esplanade-ouvert/'}

- Let's put it in a function with no argument

In [17]:
def get_all_links():
    """
    Get all restaurants links and create a dict of link with ru_names as keys
    """
    # Webpage from which we extract all links
    urlpage = 'https://www.crous-strasbourg.fr/restauration/vos-restaurants-et-leurs-menus/'
    
    # Use our get_page function to extract a list with all list
    links = get_page(urlpage,'div','links')
    
    # Create an empty dict
    names_links = {}
    
    # Iterate over each link
    for link in links:
        # Check if the restaurant is open, it is written in the url !
        if bool(re.search('ouvert',str(link))):
            # Extract the name of the restaurant
            name = re.findall('restaurant/(.*)-ouvert/',str(link))[0]
            # Extract the link
            link = re.search('http:.*ouvert/',str(link)).group()
            # Update the dict
            names_links.update({name : link})
            
    return names_links
    

- We can now make a try to get all the information for a restaurant, let's try for the 2 first days (6 meals)

In [18]:
n_days = 2
page = get_page('https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-ouvert/','ul','slides')
dates = re.findall('<h3>(.*?)</h3>',str(page))[:n_days]
restaurant = re.findall('<div class="content-repas"><div>(.*?)</div>',str(page))[:n_days*3]
restaurant

['<span class="name">Pas de service</span>',
 '<span class="name">Pôle Ici et Ailleurs</span><ul class="liste-plats"><li>UNIQUEMENT A EMPORTER</li><li></li><li>Boulettes de viande (boeuf et porc) maison</li><li>Sauce forestière</li><li>Gratin dauphinois</li><li>Brocolis</li></ul><span class="name">Pôle végétarien</span><ul class="liste-plats"><li>Colombo de légumes au lait de coco</li></ul><span class="name">Cristal Shop (RDC)</span><ul class="liste-plats"><li>UNIQUEMENT A EMPORTER</li><li></li><li>Burger du jour</li><li>Pizza du jour</li></ul><span class="name">Desserts maison</span><ul class="liste-plats"><li>Gâteau de patates douces maison</li></ul><span class="name">Pôle Pêche du jour</span><ul class="liste-plats"><li>menu non communiqué</li></ul>',
 '<span class="name">Pas de service</span>',
 '<span class="name">Pas de service</span>',
 '<span class="name">Pôle Ici et Ailleurs</span><ul class="liste-plats"><li>menu non communiqué</li></ul><span class="name">Pôle végétarien</span>

In [19]:
dates

['Menu du jeudi 28 janvier 2021', 'Menu du vendredi 29 janvier 2021']

In [20]:
for meal in restaurant:
    print(get_menu(meal))

{}
{'Pôle Ici et Ailleurs': ['Boulettes de viande (boeuf et porc) maison', 'Sauce forestière', 'Gratin dauphinois', 'Brocolis'], 'Pôle végétarien': ['Colombo de légumes au lait de coco'], 'Cristal Shop (RDC)': ['Burger du jour', 'Pizza du jour'], 'Desserts maison': ['Gâteau de patates douces maison'], 'Pôle Pêche du jour': ['menu non communiqué']}
{}
{}
{'Pôle Ici et Ailleurs': ['menu non communiqué'], 'Pôle végétarien': ['Pizza végétarienne maison', 'Salade verte'], 'Cristal Shop (RDC)': ['Burger du jour', 'Pizza du jour'], 'Desserts maison': ['Financier aux myrtilles maison'], 'Pôle Pêche du jour': ['Tartiflette irlandaise', 'Salade verte']}
{}


- It seems that everything works fine, let's put all together!

- The basis of our final step will be as follow:
   - Get all links
   - Iterate on it

In [21]:
links = get_all_links()

In [22]:
n_days = 2
page = get_page('https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade-ouvert/','ul','slides')
dates = re.findall('<h3>(.*?)</h3>',str(page))[:n_days]
restaurant = re.findall('<div class="content-repas"><div>(.*?)</div>',str(page))[:n_days*3]
for meal in restaurant:
    get_menu(meal)

- Now we just need to think about how storing it: a dict is a good solution, once each date is set we just need to update the dict with the next restaurant!

In [23]:
all_ru_menu = {}
meal_keys = ['breakfast','lunch','dinner']
for date in dates:
        if date not in all_ru_menu:
            all_ru_menu.update({date:{meal:{} for meal in meal_keys}})
all_ru_menu

{'Menu du jeudi 28 janvier 2021': {'breakfast': {}, 'lunch': {}, 'dinner': {}},
 'Menu du vendredi 29 janvier 2021': {'breakfast': {},
  'lunch': {},
  'dinner': {}}}

- Why are we doing it ? If a date is new to the dict we need to set this key before trying to update its values  
If we update a dict this way ```dict.update({key1:{key2:values}})``` we are removing everything before  in ```key1```.  
Then by creating the key before trying to update it, we can easily access to the dict to update new restaurant like this: ```dict[key1].update({key2:values})```.

- Also to have date, meal type, and restaurant menu that evolve together we can zip the three elements in order to use them in the loop.

In [24]:
print(np.repeat(dates,3).tolist())
print(meal_keys*n_days)
print(restaurant)

zip(np.repeat(dates,3).tolist(),meal_keys*n_days,restaurant)

['Menu du jeudi 28 janvier 2021', 'Menu du jeudi 28 janvier 2021', 'Menu du jeudi 28 janvier 2021', 'Menu du vendredi 29 janvier 2021', 'Menu du vendredi 29 janvier 2021', 'Menu du vendredi 29 janvier 2021']
['breakfast', 'lunch', 'dinner', 'breakfast', 'lunch', 'dinner']
['<span class="name">Pas de service</span>', '<span class="name">Pôle Ici et Ailleurs</span><ul class="liste-plats"><li>UNIQUEMENT A EMPORTER</li><li></li><li>Boulettes de viande (boeuf et porc) maison</li><li>Sauce forestière</li><li>Gratin dauphinois</li><li>Brocolis</li></ul><span class="name">Pôle végétarien</span><ul class="liste-plats"><li>Colombo de légumes au lait de coco</li></ul><span class="name">Cristal Shop (RDC)</span><ul class="liste-plats"><li>UNIQUEMENT A EMPORTER</li><li></li><li>Burger du jour</li><li>Pizza du jour</li></ul><span class="name">Desserts maison</span><ul class="liste-plats"><li>Gâteau de patates douces maison</li></ul><span class="name">Pôle Pêche du jour</span><ul class="liste-plats">

In [25]:
all_ru_menu = {}
n_days = 3
meal_keys = ['breakfast','lunch','dinner']

# Get all links for opened restaurants
links = get_all_links()

# Loop over the links, since it is a dict it with loop on the keys, wich are the names
for name in links:
    print('retrieve menu of the '+name)
    # Get the content
    page = get_page(links[name],
                    element = 'ul',
                    html_class = 'slides')
    
    # Clean dates and meals
    dates = re.findall('<h3>(.*)</h3>',str(page))[:n_days]
    restaurant = re.findall('<div class="content-repas"><div>(.*?)</div>',str(page))[:n_days*3]
    # update dict keys if the date is new
    for date in dates:
        if date not in all_ru_menu:
            all_ru_menu.update({date:{meal:{} for meal in meal_keys}})
    
    # Loop over the content and use the keys to feed the dict
    # Since there is 3 types of lunch we repeat dates thee times
    date_meal_menu = zip(np.repeat(dates,3).tolist(),meal_keys*n_days,restaurant)
    
    for date, meal, meal_menu in date_meal_menu:
        # Get the menu and clean it a bit
        menu = get_menu(meal_menu)
        
        # if the menu is not empy
        if bool(menu):
            # update the dictionary with the name of the restaurant and the menu
            all_ru_menu[date][meal].update({name:menu})
    # ZzZzz
    time.sleep(2)

print('Download finished')


retrieve menu of the resto-u-gallia
retrieve menu of the resto-u-paul-appell
retrieve menu of the resto-u-esplanade
retrieve menu of the resto-u-cronenbourg
retrieve menu of the resto-u-illkirch
retrieve menu of the resto-u-de-lillberg
retrieve menu of the resto-u-de-liut-mulhouse
retrieve menu of the resto-u-de-colmar
retrieve menu of the le-cristal-shop-ru-esplanade
Download finished


In [26]:
all_ru_menu

{'Menu du jeudi 28 janvier 2021': {'breakfast': {},
  'lunch': {'resto-u-gallia': {'Menu étudiant': ['Risotto au bœuf',
     'Escalope viennoise',
     'Semoule',
     'Légumes',
     'Pizza paysanne'],
    'Végétarien ou végétal': ['Clafoutis aux tomates cerises',
     'Salade verte',
     'Tchoutchouka'],
    'Brasserie': ['menu non communiqué']},
   'resto-u-paul-appell': {'Pôle végétal': ['Pâtes au fromage'],
    'Flam and Co': ['Paëlla'],
    'Plat du jour': ['VENTE A EMPORTER',
     '1 Entrée ou 1 Soupe',
     '1 Plat chaud',
     '1 Dessert',
     'Tarif 1€']},
   'resto-u-esplanade': {'Pôle Ici et Ailleurs': ['Boulettes de viande (boeuf et porc) maison',
     'Sauce forestière',
     'Gratin dauphinois',
     'Brocolis'],
    'Pôle végétarien': ['Colombo de légumes au lait de coco'],
    'Cristal Shop (RDC)': ['Burger du jour', 'Pizza du jour'],
    'Desserts maison': ['Gâteau de patates douces maison'],
    'Pôle Pêche du jour': ['menu non communiqué']},
   'resto-u-cronenbour

- Finally! You can easily get information from your dict

In [27]:
all_ru_menu['Menu du jeudi 28 janvier 2021']['lunch']['resto-u-gallia']

{'Menu étudiant': ['Risotto au bœuf',
  'Escalope viennoise',
  'Semoule',
  'Légumes',
  'Pizza paysanne'],
 'Végétarien ou végétal': ['Clafoutis aux tomates cerises',
  'Salade verte',
  'Tchoutchouka'],
 'Brasserie': ['menu non communiqué']}

- Now we can finish with a small user interface to make things more fancy !
- Remerber that we saw an easy way of formatting strings, create a function to print out stuff with an index aside

In [28]:
def show_item(list_):
    i = 1
    for item in list_:
        print('%s : %s'%(str(i),item))
        i+=1

- Ask the user the rest using ```input()```

In [29]:
quit = 'N'
change_rest = 'Y'
while quit=='N':
    dates = list(all_ru_menu.keys())
    show_item(dates)
    date_choosed = int(input('Select the date (type the number aside)'))
    show_item(meal_keys)
    meal_choosed = int(input('Select the meal (type the number aside)'))
    restaurants = list(all_ru_menu[dates[date_choosed-1]][meal_keys[meal_choosed-1]].keys())
    while change_rest == 'Y':
        show_item(restaurants)
        rest_choosed = int(input('Select the restaurant (type the number aside)'))
        menu = all_ru_menu[dates[date_choosed-1]][meal_keys[meal_choosed-1]][restaurants[rest_choosed-1]]
        for meal in menu:
            print('''%s :
            %s
              '''%(meal,menu[meal]))
        change_rest = input('Do you want to select another restaurant ? (Y/N)')
    
    
    quit = input('Do you want to quit ? (Y/N)')

1 : Menu du jeudi 28 janvier 2021
2 : Menu du vendredi 29 janvier 2021
3 : Menu du lundi 1 février 2021
Select the date (type the number aside)2
1 : breakfast
2 : lunch
3 : dinner
Select the meal (type the number aside)2
1 : resto-u-gallia
2 : resto-u-esplanade
3 : resto-u-cronenbourg
4 : resto-u-illkirch
5 : resto-u-de-lillberg
6 : resto-u-de-liut-mulhouse
7 : resto-u-de-colmar
8 : le-cristal-shop-ru-esplanade
Select the restaurant (type the number aside)2
Pôle Ici et Ailleurs :
            ['menu non communiqué']
              
Pôle végétarien :
            ['Pizza végétarienne maison', 'Salade verte']
              
Cristal Shop (RDC) :
            ['Burger du jour', 'Pizza du jour']
              
Desserts maison :
            ['Financier aux myrtilles maison']
              
Pôle Pêche du jour :
            ['Tartiflette irlandaise', 'Salade verte']
              
Do you want to select another restaurant ? (Y/N)Y
1 : resto-u-gallia
2 : resto-u-esplanade
3 : resto-u-cronenbourg
4 :